In [1]:
import simulationlib as sl
import lasinglib as ll
import measurelib as ml

import modulators as mo
import shapes

import numpy as np
import matplotlib.pyplot as plt

import logging
import os

In [4]:
CHIP = sl.SimGrid(dimension=32, resolution=101, thickness=0.03, use_spar=False,
                  spar_thickness=0.5, spar_width=1)

print(dir(mo))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'doubleGaussianRamp', 'doubleSinePulse', 'make_exp_pulse', 'math', 'normal_curve', 'np']
